In [106]:
import pandas as pd
from pymongo import MongoClient
import requests
import json

# API

In [132]:
data = pd.read_csv('../data/final_dataset_to_mongo.csv', index_col='Unnamed: 0')

In [133]:
data.sample(5)

,Brand,Category,Name,Skin_Type,Ingredients,URL,Price,Rating
5995,Proactiv,Cleanser,Deep Cleansing Wash,Dry,Water- Cocamidopropyl Betaine- Distearyl Phth...,https://www.sephora.com/product/deep-cleansing...,29.05,5.0
6191,SEPHORA COLLECTION,Face Oil,Multi-Purpose Oil,Dry,Vitis Vinifera (Grape) Seed Oil- Diisopropyl A...,https://www.sephora.com/product/multi-purpose-...,6.64,4.5
2076,Dr. Jart+,Face Serum,Focuspot™ Micro Tip™ Patches,Combination,Sodium Hyaluronate- Trehalose- Clitoria Ternat...,https://www.sephora.com/product/focuspot-micro...,14.94,3.0
1855,Dr. Brandt Skincare,Face Serum,Needles No More™ Wrinkle Smoothing Cream,Dry,-Neuropeptides: Deliver skin relaxing ingredi...,https://www.sephora.com/product/needles-no-mor...,73.87,3.5
4375,Lancôme,Toner,Tonique Douceur Softening Hydrating Toner with...,Combination,Water- Glycerin- Sodium Citrate- Ci 42090/ Blu...,https://www.sephora.com/product/tonique-douceu...,22.41,4.5


## Mongo queries

For example:

In [134]:
con = MongoClient("localhost:27017")

In [135]:
db = con.get_database("skincare")

In [136]:
collection = db.get_collection("first")

In [137]:
# Single query

one = list(collection.find({'Brand':'Anthony'}, {'_id':0}))

In [138]:
one[0]

{'Brand': 'Anthony',
 'Category': 'Cleanser',
 'Name': 'Glycolic Facial Cleanser',
 'Skin_Type': 'Normal',
 'Ingredients': ' -Glycolic Acid (4.9%): Works to loosen the surface layer of dead cells and helps clear oil.-Aloe Vera: Moisturizes- soothes- and promotes healing while calming skin without creating a greasy feeling.-Vitamin A: Provides antioxidant action to soften- smooth- and increase elasticity of skin.-Vitamin C: Provides antioxidant action to protect- firm- and brighten skin.\n\n\n\nWater- Cocoamidopropyl Oxide- Glycerin- Cetyl Alcohol- Stearyl Alcohol- Potassium Hydroxide- Sodium Phytate- Citrus Grandis Peel Oil- Citrus Noblis Peel Oil- Ascorbic Acid- Tocopheryl Acetate- Retinyl Acetate- Methylchloroisothiazolinone- Methylisothiazolione- Calendula Officinalis Flower Extract- Anthemis Noblis Flower Extract.',
 'URL': 'https://www.sephora.com/product/glycolic-facial-cleanser-P9038?icid2=products grid:p9038',
 'Price': 24.9,
 'Rating': 4.5}

In [139]:
# Query using an input
# SKIN TYPE

user_input = input('Please choose a brand:')

Please choose a brand:Sensitive


In [140]:
two = list(collection.find({'Skin_Type':user_input}, {'_id':0}))

In [143]:
two[0]

{'Brand': 'Algenist',
 'Category': 'Face Serum',
 'Name': 'GENIUS Ultimate Anti-Aging Vitamin C+ Serum',
 'Skin_Type': 'Sensitive',
 'Ingredients': '-Alguronic Acid: Improves the appearance of firmness which results in more toned and younger looking skin.\n-Microalgae Oil: Keeps skin hydrated up to five times longer than untreated skin over a 24-hour period.\n-Vitamin C+: Helps to reduce the appearance of discoloration and boost radiance in the skin.\n-Plant Collagen: Hydrates and conditions the skin while promoting a more youthful appearance.\n\nWater- Glycerin- Butylene Glycol- 3-O-Ethyl Ascorbic Acid- Polymethylsilsesquioxane- Sodium Citrate- Hydroxyethyl Acrylate/Sodium Acryloyldimethyl Taurate Copolymer- Niacinamide- Ethoxydiglycol- Algae Exopolysaccharides- Chlorella Protothecoides Oil- Vinyl Dimethicone/ Methicone Silsesquioxane Crosspolymer- Citric Acid- Algae Extract- Peg-12 Dimethicone- Palmitoyl Oligopeptide- Palmitoyl Tetrapeptide-7- Caprylyl Glycol- Xanthan Gum- Vegetable 

In [144]:
len(two)

62

In [167]:
# Query using an multiple input
# SKIN TYPE

user_input = ['alpyn beauty', "Kiehl's Since 1851", 'Farmacy']

In [168]:
three = list(collection.find({'Brand':{"$in": user_input}}, {'_id':0}))

In [170]:
three[9]

{'Brand': 'alpyn beauty',
 'Category': 'Cleanser',
 'Name': 'PlantGenius® Creamy Bubbling Cleanser with Fruit Enzymes & AHAs',
 'Skin_Type': 'Dry',
 'Ingredients': " -Papaya and Pomegranate: Provide gentle enzymatic exfoliation to remove dead skin cells and refine congested pores for healthy- glowing skin.                \n-Glycolic and Lactic Acid: AHAs that support natural cell turnover- smooth- and brighten skin.\n-Hyaluronic Acid and Ceramides: Hydrate and maintain the skin's moisture barrier for plumper- more supple skin.Water- Glycerin- Sodium Cocoyl Isethionate- Sodium Hyaluronate- Hamamelis Virginiana (Witch Hazel) L Methyl Cocoyl Taurate- Cetearyl Alcohol- Lactobacillus/Punica Granatum (Pomegranate) Fruit Ferment Extract- Polyacrylate-33- Sodium Cocoyl Apple Amino Acids- Dehydroacetic Acid- Benzyl Alcohol- Tromethamine- Carica Papaya (Papaya) Fruit Extract- Potassium Sorbate- Eclipta Prostrata Extract- Melia Azadirachta Leaf Extract- Moringa Pterygosperma Seed Oil- Glycolic Ac

In [232]:
# Multi query for routine

four = list(collection.find({"$and":[{'Skin_Type':'Oily'},{'Price':{"$lte":16.66}}, {'Category':'Cleanser'}]}, {'_id':0, 'Ingredients':0, 'URL':0}).limit(3))

In [233]:
four

[{'Brand': 'belif',
  'Category': 'Cleanser',
  'Name': 'Pore Cleaner Foam Cleanser',
  'Skin_Type': 'Oily',
  'Price': 16.6,
  'Rating': 4},
 {'Brand': 'Caudalie',
  'Category': 'Cleanser',
  'Name': 'Micellar Cleansing Water',
  'Skin_Type': 'Oily',
  'Price': 11.62,
  'Rating': 4},
 {'Brand': 'Clarisonic Skincare',
  'Category': 'Cleanser',
  'Name': 'Refreshing Gel Cleanser',
  'Skin_Type': 'Oily',
  'Price': 15.77,
  'Rating': 4.5}]

In [235]:
five = list(collection.find({"$and":[{'Skin_Type':'Oily'},{'Price':{"$lte":16.66}}, {'Category':'Cleanser'}]}, {'_id':0, 'Brand':0, 'Category':0, 'Name':0, 'Skin_Type':0, 'Ingredients':0, 'Price':0, 'Rating':0}).limit(3).sort('Rating', -1))

In [237]:
five[0]

{'URL': 'https://www.sephora.com/product/purity-made-simple-micellar-cleansing-water-mini-P433159?icid2=products grid:p433159'}

### Working on API options

In [145]:
data.sample(3)

,Brand,Category,Name,Skin_Type,Ingredients,URL,Price,Rating
4863,Murad,Eye Cream,Hydro-Dynamic® Ultimate Moisture For Eyes,Normal,-Collagen Support Complex: Supports skin’s r...,https://www.sephora.com/product/hydro-dynamic-...,58.10,4.0
3728,Kate Somerville,Cleanser,Gentle Daily Wash,Normal,-Lavender Essence and Natural Sage: Soothe sk...,https://www.sephora.com/product/gentle-daily-w...,29.88,4.0
3211,Herbivore,Exfoliator,Crushed Amethyst Exfoliating Body Polish,Oily,-Crushed Amethyst Gemstone: Cosmetic-grade- fi...,https://www.sephora.com/product/crushed-amethy...,36.52,4.0


In [146]:
list_brand = list(data.Brand.unique())

In [147]:
list_brand

['Algenist',
 'Alpha-H',
 'alpyn beauty',
 'AMOREPACIFIC',
 'Anastasia Beverly Hills',
 'Anthony',
 'Armani Beauty',
 'The Art of Shaving',
 'bareMinerals',
 'BeautyBio',
 'beautyblender',
 'BECCA',
 'belif',
 'Benefit Cosmetics',
 'Biossance',
 'Bobbi Brown',
 'boscia',
 'Caudalie',
 'Charlotte Tilbury',
 'Ciaté London',
 'Clarins',
 'Clarisonic Skincare',
 'CLINIQUE',
 'Conture',
 'COOLA',
 'COVER FX',
 'Deborah Lippmann',
 'DERMAFLASH',
 'Dermalogica',
 'Dior',
 "Dr Roebuck's",
 'Dr. Barbara Sturm',
 'Dr. Brandt Skincare',
 'Dr. Dennis Gross Skincare',
 'Dr. Jart+',
 'Drunk Elephant',
 'Edible Beauty',
 'Erborian',
 'Erno Laszlo',
 'Estée Lauder',
 'Eve Lom',
 'Farmacy',
 'FARSÁLI',
 'FENTY BEAUTY by Rihanna',
 'First Aid Beauty',
 'Flora + Bast',
 'FOREO',
 'Fresh',
 'Givenchy',
 'GLAMGLOW',
 'Glow Recipe',
 'goop',
 'Grande Cosmetics',
 'Guerlain',
 'Herbivore',
 'High Beauty',
 'Hourglass',
 'ILIA',
 'INC.redible',
 'Indie Lee',
 'The INKEY List',
 'innisfree',
 'Isle of Paradise

In [148]:
list_category = list(data.Category.unique())

In [149]:
list_category

['Face Serum',
 'Moisturizer',
 'Eye Cream',
 'Sunscreen & After Sun Care',
 'Face Mask',
 'Cleanser',
 'Decollete & Neck Cream',
 'Face Oil',
 'Facial Peel',
 'Exfoliator',
 'BB & CC Cream',
 'Toner',
 'Face Set',
 'Anti-Aging',
 'Facial Roller',
 'Eye Mask',
 'Face Wipe',
 'Night Cream',
 'Blemish & Acne Treatment',
 'Face Tanner',
 'Facial Cleansing Brush']

In [150]:
data.Skin_Type.unique()

array(['Normal', 'Dry', 'Combination', 'Oily', 'Sensitive', 'Mature',
       'Oil'], dtype=object)

In [151]:
mini = float(data.Price.min())

In [152]:
mini

2.49

In [153]:
data.Price.min()

2.49

In [154]:
maxi = int(data.Price.max())

In [155]:
maxi

410

In [156]:
data.Price.max()

410.85

In [157]:
data.Rating.max()

5.0

In [172]:
number = '98.65'

In [179]:
type(number)

str

In [174]:
number_number = float(number)

In [175]:
number_number

98.65

In [180]:
type(number_number)

float

In [215]:
data.Category.unique()

array(['Face Serum', 'Moisturizer', 'Eye Cream',
       'Sunscreen & After Sun Care', 'Face Mask', 'Cleanser',
       'Decollete & Neck Cream', 'Face Oil', 'Facial Peel', 'Exfoliator',
       'BB & CC Cream', 'Toner', 'Face Set', 'Anti-Aging',
       'Facial Roller', 'Eye Mask', 'Face Wipe', 'Night Cream',
       'Blemish & Acne Treatment', 'Face Tanner',
       'Facial Cleansing Brush'], dtype=object)

In [219]:
50/3

16.666666666666668

In [216]:
data[data.Category == 'Cleanser'].Price.max()

91.3

In [217]:
data[data.Category == 'Cleanser'].Price.min()

3.32

In [221]:
data[data.Category == 'Moisturizer'].Price.max()

319.55

In [222]:
data[data.Category == 'Moisturizer'].Price.min()

4.14

In [224]:
data[data.Category == 'Sunscreen & After Sun Care'].Price.max()

219.95

In [225]:
data[data.Category == 'Sunscreen & After Sun Care'].Price.min()

12.45